In [1]:
!pip install -q -U accelerate bitsandbytes git+https://github.com/huggingface/transformers.git
!pip install datasets -q
!pip install peft -q

In [1]:
import torch
from transformers import (
    AutoProcessor,
    BitsAndBytesConfig,
    PaliGemmaForConditionalGeneration
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [5]:
model_id = "google/paligemma-3b-mix-224"

In [6]:
# Use BitsAndBytes Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [7]:
# Initialise the model
model = PaliGemmaForConditionalGeneration.from_pretrained(
    pretrained_model_name_or_path=model_id, 
    quantization_config=bnb_config,
    device_map={"":0}
).eval()

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
processor = AutoProcessor.from_pretrained(model_id)

In [9]:
from PIL import Image
import requests

# Input 1: The prompt
prompt = "What is on the flower?"

# Input 2: The image
image_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/bee.jpg?download=true"
image = Image.open(requests.get(image_url, stream=True).raw)

# Encode the inputs
inputs = processor(prompt, image, return_tensors="pt")
inputs_length = inputs["input_ids"].shape[-1] 

with torch.inference_mode():
    # Perform the inference
    outputs = model.generate(**inputs, max_new_tokens=20, do_sample=False)
    output = outputs[0][inputs_length:] # filter out the prompt

    # Decode the output
    output_decoded = processor.decode(output, skip_special_tokens=True)
    print(output_decoded)

You're using a GemmaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/utils.py:1637: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


bee
